<a href="https://colab.research.google.com/github/kwanhong66/BERT-Kor-Practices/blob/master/notebooks/Fast_Bert_KorClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [ ]:
!nvidia-smi

Tue Jun 23 06:40:36 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install fast-bert

     |████████████████████████████████| 81kB 3.1MB/s 
     |████████████████████████████████| 204kB 14.1MB/s 
     |████████████████████████████████| 675kB 19.4MB/s 
     |████████████████████████████████| 3.8MB 34.5MB/s 
     |████████████████████████████████| 112kB 43.2MB/s 
     |████████████████████████████████| 890kB 36.2MB/s 
     |████████████████████████████████| 1.1MB 41.0MB/s 
     |████████████████████████████████| 552kB 39.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=03fb5aae43b10f21daca7986b885d527bd0ebeb6ae2fda43c7d4887200d1f8cc
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
!pip install progressbar

  Created wheel for progressbar: filename=progressbar-2.5-cp36-none-any.whl size=12074 sha256=c17dcd2ab637a349196dbe8c49f089b4722dd64451943615d01821857cbf8fec
  Stored in directory: /root/.cache/pip/wheels/c0/e9/6b/ea01090205e285175842339aa3b491adeb4015206cda272ff0
Successfully built progressbar


In [ ]:
import torch

import pandas as pd
import collections
import os
from tqdm import tqdm, trange
import sys
import random
import numpy as np

## Preparing datasets

### Local directory for datasets to be used

In [ ]:
import pathlib

DATA_PATH = 'data' #@param {type: 'string'}
LABEL_PATH = 'labels' #@param {type: 'string'}

data_path = pathlib.Path(DATA_PATH)
data_path.mkdir(parents=True, exist_ok=True)

labels_path = pathlib.Path(LABEL_PATH)
labels_path.mkdir(parents=True, exist_ok=True)

### Getting dataset from S3 to local

Get aws key info using `getpass` to access bucket.

In [ ]:
from getpass import getpass
key_id = getpass('Enter the access key id value: ')
secret_key = getpass('Enter the secret access key value: ')

Enter the access key id value: ··········
Enter the secret access key value: ··········


In [ ]:
import boto3
s3 = boto3.resource('s3', aws_access_key_id=key_id, aws_secret_access_key=secret_key)
bucket_name = 'YOUR_BUCKET_NAME' #@param {type: 'string'}

In [ ]:
from botocore.client import ClientError

try:
    s3.meta.client.head_bucket(Bucket=bucket_name)
    buck = s3.Bucket(bucket_name)
except ClientError:
    # The bucket does not exist or you have no access.
    print('There is no bucket {} or you have no access of it'.format(bucket_name))

Download datasets files from s3 into local directory

Each object has key for identifying it in the bucket

In [ ]:
# dataset files
train_file_key = 'train.csv' #@param {type: 'string'}
val_file_key = 'dev.csv' #@param {type: 'string'}

local_train_file_name = train_file_key
local_val_file_name = val_file_key

# buck.download_file('data/' + train_file_key, DATA_PATH + '/' + local_train_file_name)
# buck.download_file('data/' + val_file_key, DATA_PATH + '/' + local_val_file_name)

# labels file
labels_file_key = 'labels.csv' #@param {type: 'string'}
local_labels_file_name = labels_file_key

# buck.download_file('data/' + labels_file_key, LABEL_PATH + '/' + local_labels_file_name)

In [ ]:
# https://gist.github.com/wy193777/e7607d12fad13459e8992d4f69b53586
def hook(t):
  def inner(bytes_amount):
    t.update(bytes_amount)
  return inner

def download_from_s3_tqdm(bucket_name, s3_dir, s3_file_key, local_dir, local_file_name):
  # S3 bucket: 'selvasai-bert-finetuning'/data{directory}/train.csv{object_key}
  file_object = s3.Object(bucket_name, s3_dir + '/' + s3_file_key)
  filesize = file_object.content_length
  
  with tqdm(total=filesize, unit='KB', unit_scale=True, desc=s3_file_key) as t:
    buck.download_file(s3_dir + '/' + s3_file_key, local_dir + '/' + local_file_name, Callback=hook(t))

download_from_s3_tqdm(bucket_name, 'data/test', train_file_key, DATA_PATH, local_train_file_name)
download_from_s3_tqdm(bucket_name, 'data/test', val_file_key, DATA_PATH, local_val_file_name)
download_from_s3_tqdm(bucket_name, 'data/test', labels_file_key, LABEL_PATH, local_labels_file_name)

train.csv: 100%|██████████| 76.2k/76.2k [00:01<00:00, 70.4kKB/s]
dev.csv: 100%|██████████| 16.6k/16.6k [00:00<00:00, 31.0kKB/s]
labels.csv: 100%|██████████| 4.00/4.00 [00:00<00:00, 7.42KB/s]


### BertDataBunch

In [ ]:
from fast_bert.data_cls import BertDataBunch

databunch = BertDataBunch(DATA_PATH, LABEL_PATH,
                          tokenizer='monologg/kobert',
                          train_file=local_train_file_name,
                          val_file=local_val_file_name,
                          label_file=local_labels_file_name,
                          text_col='sentences',
                          label_col='labels',
                          batch_size_per_gpu=8,
                          max_seq_length=512,
                          multi_gpu=False,
                          multi_label=False,
                          model_type='bert')

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils.py:831: FutureWarning: Parameter max_len is deprecated and will be removed in a future release. Use model_max_length instead.
  category=FutureWarning,


#### File format for train.csv and val.csv


| index | text                                                                                                                                                                                                                                                                                                                                | label |
| ----- | ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | ----- |
| 0     | Looking through the other comments, I'm amazed that there aren't any warnings to potential viewers...                                             | neg   |
| 1     | I've watched the first 17 episodes and this series is simply amazing! I haven't been this interested in... | pos   |
| 2     | his movie is nothing short of a dark, gritty...                                                                                                                                                                                         | pos   |

In case the column names are different than the usual text and labels, you will have to provide those names in the databunch `text_col` and `label_col `parameters.

## Model setup

In [ ]:
OUTPUT_DIR = 'output' #@param {type: 'string'}

output_path = pathlib.Path(OUTPUT_DIR)
output_path.mkdir(parents=True, exist_ok=True)

NameError: ignored

### BertLearner

In [ ]:
from fast_bert.learner_cls import BertLearner
from fast_bert.metrics import accuracy
import logging

logger = logging.getLogger()
device_cuda = torch.device("cuda")
metrics = [{'name': 'accuracy', 'function': accuracy}]

learner = BertLearner.from_pretrained_model(
						databunch,
						pretrained_path='monologg/kobert',
						metrics=metrics,
						device=device_cuda,
						logger=logger,
						output_dir=OUTPUT_DIR,
						finetuned_wgts_path=None,
						warmup_steps=50,
						multi_gpu=False,
						is_fp16=False,
						multi_label=False,
						logging_steps=50)

NameError: ignored

### Tensorboard

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --inspect --logdir /content/output/tensorboard/

ERROR: Failed to launch TensorBoard (exited with 0).
Contents of stderr:
2020-06-23 06:05:40.520864: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Contents of stdout:
Processing event files... (this can take a few minutes)

Found event files in:
/content/output/tensorboard/

These tags are in /content/output/tensorboard/:
audio -
histograms -
images -
scalars
   eval_accuracy
   eval_loss
   loss
   lr
tensor -

Event statistics for /content/output/tensorboard/:
audio -
graph -
histograms -
images -
scalars
   first_step           50
   last_step            750
   max_step             750
   min_step             50
   num_steps            15
   outoforder_steps     []
sessionlog:checkpoint -
sessionlog:start -
sessionlog:stop -
tensor -

## Train the model

In [ ]:
EPOCHS = 5 #@param {type: "number"}
SCHEDULE_TYPE = 'warmup_cosine' #@param ["warmup_cosine", "warmup_linear", "warmup_constant", "warmup_cosine_hard_restarts", "warmup_cosine_warmup_restarts", "none"]
OPTIMIZER_TYPE = 'lamb' #@param ["lamb", "adamw"]

In [ ]:
learner.fit(epochs=EPOCHS,
			lr=6e-5,
			validate=True, 	# Evaluate the model after each epoch
			schedule_type=SCHEDULE_TYPE,
			optimizer_type=OPTIMIZER_TYPE)

/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


(750, 0.19511465267340342)

## Save trained model artifacts

In [ ]:
learner.save_model()

Model artefacts will be persisted in the output_dir/'model_out' path provided to the learner object. Following files will be persisted:

| File name               | description                                      |
| ----------------------- | ------------------------------------------------ |
| pytorch_model.bin       | trained model weights                            |
| spiece.model            | sentence tokenizer vocabulary (for xlnet models) |
| vocab.txt               | workpiece tokenizer vocabulary (for bert models) |
| special_tokens_map.json | special tokens mappings                          |
| config.json             | model config                                     |
| added_tokens.json       | list of new tokens                               |

As the model artefacts are all stored in the same folder, you will be able to instantiate the learner object to run inference by pointing pretrained_path to this location.

## Model inference

In [ ]:
texts = ['완전 노잼', '재밌었다']
predictions = learner.predict_batch(texts)
print(predictions)

NameError: ignored

In [ ]:
from fast_bert.prediction import BertClassificationPredictor

MODEL_PATH = OUTPUT_DIR + 'model_out'

predictor = BertClassificationPredictor(
				model_path=MODEL_PATH,
				label_path=LABEL_PATH, # location for labels.csv file
				multi_label=False,
				model_type='bert',
				do_lower_case=False)

# Single prediction
prediction_text = "완전 핵노잼" #@param {type: "string"}
single_prediction = predictor.predict(prediction_text)
print(single_prediction)

# Batch predictions
texts = [
	"볼만했음",
	"감동 그 자체"
	]

multiple_predictions = predictor.predict_batch(texts)
print(multiple_predictions)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils.py:831: FutureWarning: Parameter max_len is deprecated and will be removed in a future release. Use model_max_length instead.
  category=FutureWarning,


[('0', 0.9716218709945679), ('1', 0.028378207236528397)]
[[('0', 0.9711145758628845), ('1', 0.028885483741760254)], [('0', 0.9717930555343628), ('1', 0.028206944465637207)]]


## Upload fine-tuned model artifacts (Optional for better serving workflow)

After fine-tuning model, artifacts are stored into `OUTPUT_DIR/model_out`.

We can upload these artifacts to served in other framework or application.

In [ ]:
def upload_to_aws(local_file, s3_file):

    statinfo = os.stat(local_file)
    up_progress = progressbar.progressbar.ProgressBar(maxval=statinfo.st_size)
    up_progress.start()
    def upload_progress(chunk):
        up_progress.update(up_progress.currval + chunk)

    try:
        buck.upload_file(local_file, s3_file, Callback=upload_progress)
        up_progress.finish()
        print("Upload Successful")
        return True
    except FileNotFoundError:
        print("The file was not found")
        return False
    except NoCredentialsError:
        print("Credentials not available")
        return False

In [ ]:
# https://stackoverflow.com/questions/43399544/uploading-a-folder-full-of-files-to-a-specific-folder-in-amazon-s3
local_folder = OUTPUT_DIR + '/' + 'model_out'
s3_dir = 'model_output/test'
for root, dirs, files in os.walk(os.path.abspath(local_folder)):
    for filename in files:
        file_key_name = s3_dir + '/' + filename
        local_name = os.path.join(root, filename)
        print(local_name)
        buck.upload_file(local_name, file_key_name)

/content/output/model_out/test.txt
/content/output/model_out/test2.txt
